In [2]:
import torch
import timm

import numpy as np
import PIL.Image as Image

from torch.utils.data import DataLoader, Dataset
from src.util.sam import SAM
from pytorch_lightning import LightningModule, Trainer

# Config

In [3]:
bs = 1

# Data

In [4]:
class GravWaveDataset(Dataset):
    
    def __init__(self, data, data_dir):
        self.data = data
        self.data_dir = data_dir
        
        self.images = [np.array(Image.open("{}/{}/{}.jpg".format(data_dir, target, id_))).transpose((2, 0, 1))
              for id_, target in data]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        img = self.images[index].astype(np.float32) / 255
        
        img = torch.tensor(img)
        
        print(img.shape)
        
        return img, torch.tensor(self.data[index][1])
        

In [5]:
data_dir = "./data"

train_data = np.load("./data/train_info.npy", allow_pickle=True)
val_data = np.load("./data/validation_info.npy", allow_pickle=True)

train_ds = GravWaveDataset(train_data, "./data/train")
val_ds = GravWaveDataset(val_data, "./data/validation")

train_dl = DataLoader(train_ds, shuffle=True, batch_size=bs)
val_dl = DataLoader(val_ds, batch_size=bs)

# Lightning Module

In [6]:
class GravModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model('tf_efficientnet_l2_ns_475', pretrained=True, num_classes=1)

    def configure_optimizers(self):
        base_optimizer = torch.optim.Adam
        optimizer = SAM(self.model.parameters(), base_optimizer, lr=0.1)
#         optimizer = torch.optim.Adam(self.model.parameters(), lr=0.1)
        return optimizer
    
    def forward(self, x):
        return torch.sigmoid(self.model(x))
    
    def training_step(self, batch, batch_idx):
        print(batch)
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss
    
    def validation_step(self, batch, batch_idx):
        print(batch)
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

In [7]:
module = GravModel()

trainer = Trainer()

trainer.fit(module, train_dl)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/anaconda3/envs/GravWave/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:101: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  rank_zero_warn(f"you defined a {step_name} but have no {loader_name}. Skipping {stage} loop")

  | Name  | Type         | Params
---------------------------------------
0 | model | EfficientNet | 474 M 
---------------------------------------
474 M     Trainable params
0         Non-trainable params
474 M     Total params
1,899.239 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/anaconda3/envs/GravWave/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: -1it [00:00, ?it/s]

torch.Size([3, 768, 768])
torch.Size([3, 768, 768])


RuntimeError: stack expects a non-empty TensorList

In [7]:
torch.nn.Sigmoid

torch.nn.modules.activation.Sigmoid

In [19]:
img = Image.open("processed_test.jpg")
print(torch.tensor(img))

RuntimeError: Could not infer dtype of JpegImageFile

In [46]:
torch.tensor([]).shape

torch.Size([0])